In [20]:
from keras.datasets import imdb
from keras.preprocessing import sequence
import tensorflow as tf
import os
import keras
import numpy as np
VOCAB_SIZE = 88584
MAXLEN = 250
BATCH_SIZE =64
(train_data,train_labels),(test_data,test_labels) = imdb.load_data(num_words=VOCAB_SIZE)

In [10]:
train_labels[2]

0

In [11]:
#preprocessing, reduce/increase reviews to 250 words
train_data=sequence.pad_sequences(train_data,MAXLEN)
test_data=sequence.pad_sequences(test_data,MAXLEN)

In [12]:
#create the model
model=tf.keras.Sequential([
    tf.keras.layers.Embedding(VOCAB_SIZE,32),
    tf.keras.layers.LSTM(32),
    tf.keras.layers.Dense(1,activation="sigmoid")
])

In [13]:
model.summary()

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding (Embedding)        (None, None, 32)          2834688   
_________________________________________________________________
lstm (LSTM)                  (None, 32)                8320      
_________________________________________________________________
dense (Dense)                (None, 1)                 33        
Total params: 2,843,041
Trainable params: 2,843,041
Non-trainable params: 0
_________________________________________________________________


In [15]:
#training the model
model.compile(loss="binary_crossentropy",optimizer="rmsprop",metrics=['acc'])
history=model.fit(train_data,train_labels,epochs=10,validation_split=0.2)

Epoch 1/10
625/625 [==============================] - 66s 97ms/step - loss: 0.4241 - acc: 0.8082 - val_loss: 0.3165 - val_acc: 0.8766
Epoch 2/10
625/625 [==============================] - 41s 65ms/step - loss: 0.2381 - acc: 0.9098 - val_loss: 0.2993 - val_acc: 0.8798
Epoch 3/10
625/625 [==============================] - 41s 66ms/step - loss: 0.1849 - acc: 0.9338 - val_loss: 0.3278 - val_acc: 0.8830
Epoch 4/10
625/625 [==============================] - 50s 80ms/step - loss: 0.1510 - acc: 0.9473 - val_loss: 0.3175 - val_acc: 0.8946
Epoch 5/10
625/625 [==============================] - 81s 129ms/step - loss: 0.1287 - acc: 0.9556 - val_loss: 0.2945 - val_acc: 0.8954
Epoch 6/10
625/625 [==============================] - 78s 125ms/step - loss: 0.1121 - acc: 0.9614 - val_loss: 0.2951 - val_acc: 0.8876
Epoch 7/10
625/625 [==============================] - 76s 122ms/step - loss: 0.1003 - acc: 0.9671 - val_loss: 0.3346 - val_acc: 0.8870
Epoch 8/10
625/625 [==============================] - 76s 1

In [17]:
results=model.evaluate(test_data,test_labels)
print(results)

782/782 [==============================] - 41s 53ms/step - loss: 0.5528 - acc: 0.8545
[0.5527824759483337, 0.8545200228691101]


In [21]:
#making predictions
word_index=imdb.get_word_index()

def encode_text(text):
    tokens= keras.preprocessing.text.text_to_word_sequence(text)
    tokens= [word_index[word] if word in word_index else 0 for word in tokens]
    return sequence.pad_sequences([tokens],MAXLEN)[0]
text="that movie could have been better than it was"
encoded=encode_text(text)
print(encoded)

[  0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0
   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0
   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0
   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0
   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0
   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0
   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0
   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0
   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0
   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0
   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0
   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0
   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0
   0   0   0   0   0   0   0  12  17  97  25  74 12

In [23]:
reverse_word_index = {value:key for (key,value) in word_index.items()}

def decode_integers(integers):
    PAD=0
    text=""
    for num in integers:
        if num!= PAD:
            text+=reverse_word_index[num]+" "
    return text[:-1]
print(decode_integers(encoded))

that movie could have been better than it was


In [37]:
#make a prediction
def predict(text):
    encoded_text=encode_text(text)
    pred=np.zeros((1,250))
    pred[0]=encoded_text
    result=model.predict(pred)
    print(result[0])
    
positive_review = "Amazing movie. Loved it, watching again"
predict(positive_review)
negative_review = "Worst movie ever"
predict(negative_review)

[0.74962604]
[0.6098608]
